# L2ETS Study 2


Save data from ETS as feathers so can explore in R

## Import and save gensim model

In [1]:
import gensim 
import gensim.models as g
print(gensim.__version__)
import pandas as pd
import numpy as np
#import feather as f
import pickle
#import feather-format
import os

3.4.0


In [ ]:
mod = g.Doc2Vec.load("../../data/raw/models/all.model")

There are 38500 essays with vectors of length 200.

In [ ]:
print("\nshape=" + str(mod.docvecs.doctag_syn0.shape))

# Doc vectors

### Save doc vector indices so can merge with meta data

In [ ]:
all_doct_tags = pd.DataFrame()
doctag_ids_for_R = all_doct_tags.from_dict(mod.docvecs.doctags).T
doctag_ids_for_R.columns = ['offset', 'word_count', 'doc_count']
doctag_ids_for_R.index.name = 'essay_id'
doctag_ids_for_R.reset_index(inplace = True)
f.write_dataframe(doctag_ids_for_R, "../../data/doctag_indices_all_model.feather")

### Save doc vectors

In [ ]:
docvecs_for_R = pd.DataFrame(mod.docvecs.doctag_syn0)
f.write_dataframe(docvecs_for_R, "../../data/docvecs_all_model.feather")

In [ ]:
docvecs_for_R

##### Do the same thing as above, but as loop...

In [ ]:
#directory_string = "../../data/raw/models/L1_high_low_models/"
directory_string = "../../data/raw/models/random/"
directory = os.fsencode(directory_string)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    
    mod = g.Doc2Vec.load(directory_string+filename)
    print("\nshape=" + str(mod.docvecs.doctag_syn0.shape))
    
    all_doct_tags = pd.DataFrame()
    doctag_ids_for_R = all_doct_tags.from_dict(mod.docvecs.doctags).T
    doctag_ids_for_R.columns = ['offset', 'word_count', 'doc_count']
    doctag_ids_for_R.index.name = 'essay_id'
    doctag_ids_for_R.reset_index(inplace = True)
    #f.write_dataframe(doctag_ids_for_R, "../../data/processed/L1_high_low_models/doctag_indices_" + filename + ".feather")
    #f.write_dataframe(doctag_ids_for_R, "../../data/processed/L1_models/doctag_indices_" + filename + ".feather")

    docvecs_for_R = pd.DataFrame(mod.docvecs.doctag_syn0)
    #f.write_dataframe(docvecs_for_R,"../../data/processed/L1_high_low_models/docvecs_" + filename + ".feather")
    #f.write_dataframe(docvecs_for_R,"../../data/processed/L1_models/docvecs_" + filename + ".feather")        

## Save mapping of essay ids to low vs. high group

In [ ]:
directory_string = "../../data/raw/models/L1_models/"
directory = os.fsencode(directory_string)

essay_group_mappings = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    mod = g.Doc2Vec.load(directory_string+filename)
    print("\nshape=" + str(mod.docvecs.doctag_syn0.shape))

    all_doct_tags = pd.DataFrame()
    doctag_ids_for_R = all_doct_tags.from_dict(mod.docvecs.doctags).T
    doctag_ids_for_R.columns = ['offset', 'word_count', 'doc_count']
    doctag_ids_for_R.index.name = 'essay_id'
    doctag_ids_for_R.reset_index(inplace = True)
    doctag_ids_for_R['group'] = "low_" + filename[0:3]

    # append to dataframe
    essay_group_mappings = pd.concat([essay_group_mappings, doctag_ids_for_R])

# write dataframe for this language pair
f.write_dataframe(essay_group_mappings, "/Users/mollylewis/Documents/research/Projects/L2ETS/studies/study2/data/processed/models/low_essay_mappings.feather")

# Word Vectors

There are  24713 words with 200-D vectors. (all model)

In [ ]:
mod.wv.syn0.size/200, len(mod.wv.index2word)

Write word vecs to feather

In [ ]:
wordvecs_for_R = pd.DataFrame(mod.wv.syn0)
f.write_dataframe(wordvecs_for_R, "../../data/processed/models/all_model/wordvecs_all_model.feather")

Write word vec words to feather

In [ ]:
all_words = pd.DataFrame(mod.wv.index2word)
f.write_dataframe(all_words, "../../data/processed/models/all_model/words_all_model.feather")

#### Now do this in a loop.... (for L1 models and low/high L1 models)

In [ ]:
#directory_string = "../../data/raw/models/L1_high_low_models/"
directory_string = "../../data/raw/models/L1_models/"
directory = os.fsencode(directory_string)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    
    mod = g.Doc2Vec.load(directory_string+filename)
    
    wordvecs_for_R = pd.DataFrame(mod.wv.syn0)
    #f.write_dataframe(wordvecs_for_R, "../../data/processed/models/L1_high_low_models/wordvecs_" + filename + ".feather")   
    f.write_dataframe(wordvecs_for_R, "../../data/processed/models/L1_models/wordvecs_" + filename + ".feather")   
    
    all_words = pd.DataFrame(mod.wv.index2word)
    #f.write_dataframe(all_words, "../../data/processed/models/L1_high_low_models/words_" + filename + ".feather")
    f.write_dataframe(all_words, "../../data/processed/models/L1_models/words_" + filename + ".feather")

# Word counts


In [ ]:
wc = pd.read_pickle(open("../../data/raw/essay_word_counts.pkl", 'rb'))

In [ ]:
#wc.keys()
ec = []
for key in wc:
    df = wc[key]
    df["essay_id"] = key
    ec.append(df)

ec_df = pd.concat(ec)
feather.write_dataframe(ec_df, 'essay_word_counts.feather')